In [ ]:
import pandas as pd
import datetime
import numpy as np

field = 'solar'
if field == 'wind':
    data_folder = "/Volumes/WD-DATA-100/power_optimisation_system/raw_data/data_wind/"
    metadata = pd.read_csv('wind_metadata_filtered.csv')
    field_name = 'mean_wind_speed'

if field == 'solar':
    data_folder = "/Volumes/WD-DATA-100/power_optimisation_system/raw_data/data_solar/"
    metadata = pd.read_csv('solar_metadata_filtered.csv')
    field_name = 'glbl_irad_amt'
    
df_qs = pd.DataFrame(columns=['rad','lat','lon'])  

for idx in metadata.index:
    site, county = metadata['site'][idx], metadata['county'][idx]
    site_file = data_folder + county + '/' + site + '/2001-2020.csv'
    df_site = pd.read_csv(site_file)
    df_site['time'] = pd.to_datetime(df_site['time'])
    df_site = df_site.set_index('time')
    df_site = df_site.resample('d').mean()
    y = df_site[field_name].groupby(df_site.index.dayofyear).mean()
    x = pd.date_range(start=datetime.datetime(2020,1,1), end=datetime.datetime(2020,12,31,23,59), freq='d')
    df = pd.DataFrame()
    df['time'] = x
    df['rad'] = y
    df['lat'] = metadata['lat'][idx]
    df['lon'] = metadata['lon'][idx]
    df = df.set_index('time')
    df = df.resample('Q').mean()
    df_qs = pd.concat([df_qs, df])
    
df_q1 = df_qs[df_qs.index == datetime.datetime(2020,3,31)]
df_q2 = df_qs[df_qs.index == datetime.datetime(2020,6,30)]
df_q3 = df_qs[df_qs.index == datetime.datetime(2020,9,30)]
df_q4 = df_qs[df_qs.index == datetime.datetime(2020,12,31)]

df_q1.to_csv(field+'_climatology_q1.csv')
df_q2.to_csv(field+'_climatology_q2.csv')
df_q3.to_csv(field+'_climatology_q3.csv')
df_q4.to_csv(field+'_climatology_q4.csv')

print(df_q1)
print(df_q2)